# Sensor Based Activity Recoginition 
Challenge: cdl1 - Sensor based Activity Recognition  
Team: Lea Bütler, Manjavy Kirupa, Etienne Roulet, Si Ben Tran  

Aufgabe: DL Modell erstellen

Hier in diesem Notebook erstellen wir unsere Deep Learning Modelle.

In [19]:
import tensorflow as tf
from sklearn.model_selection import KFold
import numpy as np
import logging
from datetime import datetime
import dataclasses
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection as ms
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np

tf.debugging.set_log_device_placement(False)

# datetime as filename for logging
now = datetime.now()
date_time_string = now.strftime("%Y-%m-%d_%H-%M-%S")

logging.basicConfig(level=logging.INFO, filename = f"{date_time_string}.txt", filemode='a')
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [20]:
from dataclasses import dataclass
# Static Parameters
@dataclass
class Parameters():
    batch_size: int = 128
    epochs: int = 10
    verbosity: str = "1"
    step_size: int = 374
    number_folds: int = 2
    output_size: int = 6

In [21]:
# read the CSV file into a DataFrame
df = pd.read_csv("Alle_Messungen_trimmed.csv") 
df.head(1)

,Unnamed: 0,time,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z,Orientation_qx,Orientation_qy,Orientation_qz,id,user,class,id_combined
0,1,2023-02-27 15:02:17.156,0.0,9.722,1.278,0.0,0.0,0.0,0.0,0.0,0.0,-0.21775,-0.719579,0.631111,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen


In [22]:
# convert the string time column to datetime
epoch = pd.Timestamp('1970-01-01')

df['time'] = pd.to_datetime(df['time'])
df['time'] = (df['time'] - epoch).apply(lambda x: int(x.total_seconds() * 1000))

In [23]:
# drop the columns that are not needed
df = df.drop(columns=["id", "user", "id_combined"])
df.drop(['Unnamed: 0'], axis=1, inplace=True)
# get all types of the df
#df['time'] = pd.to_datetime(df['time'])
#df['time'] = df['time'].astype('int64')//1e9
le = LabelEncoder()
df["class"] = le.fit_transform(df["class"])


In [24]:
# without rolling windows but with timesteps
X = df.values[:, 1:13]
y = df.values[:, 13]

# Reshape X to 3D format (samples, timesteps, features)
timesteps = 1  # You can choose a different number of timesteps based on the nature of your data
n_features = X.shape[1]
X = X.reshape(-1, timesteps, n_features)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=6)
y_test = to_categorical(y_test, num_classes=6)
# calculate_rolling_stats

In [25]:
# Set the window size and step size
window_size = 5
step_size = 1

# Reshape X to 2D format (samples, features)
X = df.values[:, 1:13]

# Define y
y = df["class"].values

# Create a sliding window of X with the specified window and step sizes
X_windows = np.array([X[i:i+window_size,:] for i in range(0,X.shape[0]-window_size+1,step_size)])

# Reshape X_windows to 3D format (samples, timesteps, features)
timesteps = X_windows.shape[1]
n_features = X_windows.shape[2]
X_windows = X_windows.reshape(-1, timesteps, n_features)

# Create the corresponding y labels for the sliding windows
y_windows = np.array([y[i+window_size-1] for i in range(0,X.shape[0]-window_size+1,step_size)])
y_windows = to_categorical(y_windows, num_classes=6)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_windows, y_windows, test_size=0.2, random_state=42)


In [26]:
# log it with logging
logging.info(f"X.shape: {X.shape}, y.shape: {y.shape}, X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}, X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}")

In [27]:
# Load data and preprocess
# split train dataset into x_train and y_train
# make temporary pointer to the data cause i dont like the uppercase
x_train = X_train
x_test = X_test
y_train = y_train
y_test = y_test
    

# Something like this as first Model
def create_model_1():
    model = tf.keras.Sequential([
        # Add a 1D convolutional layer
        tf.keras.layers.Conv1D(filters=128, kernel_size=2, activation='relu', padding='same', input_shape=(timesteps, n_features)),
        
        # Add LSTM layer
        tf.keras.layers.LSTM(100),

        # Add a dense output layer
        tf.keras.layers.Dense(6, activation='softmax')  # Change activation function based on the nature of the output
    ])
    model.compile(optimizer='adam',   loss='categorical_crossentropy', metrics=['accuracy'])  # Change the loss function based on the nature of the output
    return model


# Something like this as second Model
def create_model_2():
    model = tf.keras.Sequential([
        # Add a 1D convolutional layer
        tf.keras.layers.Conv1D(filters=256, kernel_size=2, activation='relu', padding='same', input_shape=(timesteps, n_features)),
        
        # Add LSTM layer
        tf.keras.layers.LSTM(100),

        # Add a dense output layer
        tf.keras.layers.Dense(6, activation='softmax')  # Change activation function based on the nature of the output
    ])
    model.compile(optimizer='adam',   loss='categorical_crossentropy', metrics=['accuracy'])  # Change the loss function based on the nature of the output
    return model

# Something like this as Third Model
def create_model_3():
    model = tf.keras.Sequential([
        # Add a 1D convolutional layer
        tf.keras.layers.Conv1D(filters=64, kernel_size=4, activation='relu', padding='same', input_shape=(timesteps, n_features)),
        
        # Add LSTM layer
        tf.keras.layers.LSTM(100),

        # Add a dense output layer
        tf.keras.layers.Dense(6, activation='softmax')  # Change activation function based on the nature of the output
    ])
    model.compile(optimizer='adam',   loss='categorical_crossentropy', metrics=['accuracy'])  # Change the loss function based on the nature of the output
    return model

def create_model_4():
    model = tf.keras.Sequential([
        # Add a 1D convolutional layer
        tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', padding='same', input_shape=(timesteps, n_features)),
        
        # Add LSTM layer
        tf.keras.layers.LSTM(100),

        # Add a dense output layer
        tf.keras.layers.Dense(6, activation='softmax')  # Change activation function based on the nature of the output
    ])
    model.compile(optimizer='adam',   loss='categorical_crossentropy', metrics=['accuracy'])  # Change the loss function based on the nature of the output
    return model

best_model_history = None  # Keep track of the best model's history

model_histories = []
# Perform cross-validation
models = [create_model_1,create_model_2,create_model_3,create_model_4]
best_model = None
num_folds = Parameters.number_folds
# Shuffel the Data with the Folds
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_acc_scores = []

for i, (train, test) in enumerate(kfold.split(x_train, y_train)):
    logging.info(f'Fold {i+1}')
    train_x, train_y = x_train[train], y_train[train]
    test_x, test_y = x_train[test], y_train[test]
    
    fold_histories = []

    
    for j, model_creator in enumerate(models):
        model = model_creator()
        logging.info(f'Model {j+1}')
        history = model.fit(train_x, train_y, epochs=Parameters.epochs, batch_size=Parameters.batch_size, validation_data=(test_x, test_y), verbose=Parameters.verbosity)
        test_loss, acc = model.evaluate(test_x, test_y, verbose=Parameters.verbosity)
        logging.info(f'Validation accuracy: {acc}')
        
        fold_histories.append(history.history)
        
        for epoch in range(Parameters.epochs):
            # Log accuracy after each epoch
            acc_epoch = history.history['val_accuracy'][epoch]
            logging.info(f'Epoch {epoch + 1}, Validation accuracy: {acc_epoch}')
        fold_acc_scores.append((i, j, acc))
        
        if best_model_history is None or acc > best_model_acc:
            best_model_history = history
            best_model = model  # Store the trained model instance
            best_model_acc = acc
    
    model_histories.append(fold_histories)

# Find the best model
best_model_index = np.argmax([score[2] for score in fold_acc_scores])
best_fold_idx, best_model_idx, _ = max(fold_acc_scores, key=lambda x: x[2])
best_model_history = model_histories[best_fold_idx][best_model_idx]
# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(x_test, y_test)
logging.info(f'Test accuracy {test_acc}')

Epoch 1/10
Epoch 2/10


In [ ]:
model_histories

In [ ]:
# Manual Test with unseen Data Future it should be the webapp
df_yvo = df_yvo.drop(columns=["id", "user", "id_combined"])
df_yvo.drop(['Unnamed: 0'], axis=1, inplace=True)
df['time'] = pd.to_datetime(df['time'])
df['time'] = (df['time'] - epoch).apply(lambda x: int(x.total_seconds() * 1000))
# get all types of the df
X = df_yvo.values[:, 1:13]
print(X)


# Create a sliding window of X with the specified window and step sizes
X_windows = np.array([X[i:i+window_size,:] for i in range(0,X.shape[0]-window_size+1,step_size)])

# Reshape X_windows to 3D format (samples, timesteps, features)
timesteps = X_windows.shape[1]
n_features = X_windows.shape[2]
print(n_features)
X_windows = X_windows.reshape(-1, timesteps, n_features)
X_windows = X_windows.astype('float32')

# Create the corresponding y labels for the sliding windows

# Split the dataset into training and testing sets

y_pred = best_model.predict(X_windows)
# Get the predicted class probabilities for each input window
y_pred_probs = best_model.predict(X_windows)

# Get the predicted class labels for each input window
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Print the predicted class labels
# get the median of the predicted labels
sol = np.median(y_pred_labels)
sol

In [ ]:
# visualize the training process
import matplotlib.pyplot as plt
import numpy as np

print(best_model_history['loss'])
# summarize history for accuracy
plt.plot(best_model_history['accuracy'])
plt.plot(best_model_history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(best_model_history['loss'])
plt.plot(best_model_history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarise best model
best_model.summary()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
#for multiclass classification
y_pred = best_model.predict(x_test)
y_test_labels = y_test.argmax(axis=1)
y_pred_labels = y_pred.argmax(axis=1)

cm = confusion_matrix(y_test_labels, y_pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
# get f1 score of each class
from sklearn.metrics import f1_score
f1 = f1_score(y_test_labels, y_pred_labels, average=None)
# get f1 score of each class

disp.plot()
plt.show()
display(f1)

In [ ]:
# Create a Baseline Model

In [ ]:
# Compare Baseline vs best_model

In [ ]:
# plot graph for learning curve and loss curve from baseline and best_model

In [ ]:
# print loss and accuracy for best model over epoch and steps plot

In [ ]:
# Export Model for using in tensorflow.js
!mkdir -p saved_model
best_model.save('saved_model/sensor_model')
best_model.save('saved_model/sensor_model.h5')

In [ ]:
# Save the weight for the Js Model
best_model.save_weights('./checkpoints/my_checkpoint')


In [ ]:
# upload model to server to download it on tensorflow js

In [ ]:
dill.dump_session('notebook_env.db')